In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score, auc, roc_auc_score, confusion_matrix
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('drive/MyDrive/CS247/train_emb.pkl', 'rb') as fid:
     train_data= pickle.load(fid)

with open('drive/MyDrive/CS247/val_emb.pkl', 'rb') as fid:
     valid_data= pickle.load(fid)

with open('drive/MyDrive/CS247/test_emb.pkl', 'rb') as fid:
     test_data= pickle.load(fid)

with open('drive/MyDrive/CS247/train_data.pkl', 'rb') as fid:
     comment_train_data= pickle.load(fid)

with open('drive/MyDrive/CS247/test_data.pkl', 'rb') as fid:
     comment_test_data= pickle.load(fid)

In [ ]:
## GETTING LIST FROM THE LOADED DATA
train_data = list(train_data)
valid_data = list(valid_data)
test_data = list(test_data)

In [ ]:
## GENERATE PANDAS DATAFRAMES
train_df = pd.DataFrame(train_data)
valid_df = pd.DataFrame(valid_data)
test_df = pd.DataFrame(test_data)

train_df.head(10)

,0,1,2,3,4,5,6
0,"[0.2640609774222722, -0.4600453673251684, 0.03...",0,0,0,0,0,0
1,"[-0.04011194184194853, -0.5940424284618303, -0...",1,0,1,1,0,0
2,"[0.3461568960768919, -0.5365164560302055, 0.11...",0,0,0,0,0,0
3,"[0.22332606270303804, -0.3382712470869757, 0.0...",0,0,0,0,0,0
4,"[0.325931273794575, -0.20956713868307272, 0.14...",0,0,0,0,0,0
5,"[1.183401224224616, -0.5341410448336582, 0.360...",0,0,0,0,0,0
6,"[0.5360300184813922, -0.4562388682939364, 0.49...",0,0,0,0,0,0
7,"[0.5236535839338992, -0.8367316260505495, 0.75...",0,0,0,0,0,0
8,"[0.27385612539126286, -0.273880081400397, 0.08...",0,0,0,0,0,0
9,"[0.3366081304893847, -0.2594793260630475, 0.01...",0,0,0,0,0,0


In [ ]:
## CHECK DATA STRUCTURE

comment_train_data.head(1)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,Target,comment_text_wp,comment_tokenize,comment
112366,591e7ceac4e56588,Would agree. While it's clear that GQ is ripe ...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]",would agree clear gq ripe trek references refe...,"[would, agree, clear, gq, ripe, trek, referenc...",would agree while it is clear that gq is ripe ...


In [ ]:
## CHECK HOW MANY EXAMPLES PER CLASS
toxic = valid_df[1].sum(0)
severe_toxic = valid_df[2].sum(0)
obscene = valid_df[3].sum(0)
threat = valid_df[4].sum(0)
insult = valid_df[5].sum(0)
identity_hate = valid_df[6].sum(0)

print("Toxic examples: ", toxic)
print("Severe toxic examples: ", severe_toxic)
print("Obscene examples: ", obscene)
print("Threat examples: ", threat)
print("Insult examples: ", insult)
print("Identity hate examples: ", identity_hate)

Toxic examples:  5397
Severe toxic examples:  557
Obscene examples:  2811
Threat examples:  3007
Insult examples:  164
Identity hate examples:  497


In [ ]:
## CLASSIFIERS

def basic_classifier_tester(X_train,Y_train,X_test,Y_test,class_weight=None,use_gaussian=False):
    f1 = []
    report_scores = {}
    if(not use_gaussian):
      classifiers = [LogisticRegression(max_iter=200,class_weight=class_weight), BernoulliNB()]
    else:
      classifiers = [LogisticRegression(max_iter=200,class_weight=class_weight), GaussianNB()]
    for classifier in (classifiers):
        classifier.fit(X_train, Y_train)
        predictions = classifier.predict(X_test)
        f1.append(f1_score(Y_test, predictions))
        report = classification_report(Y_test, predictions)
        report_scores[classifier.__class__.__name__] = report
        print(classifier.__class__.__name__ + ":")
        print(report)
        print("Confusion Matrix")
        print(confusion_matrix(Y_test, predictions))
        print('------------------------------------------------------')
    return report_scores, f1

## USEFUL FUNCTIONS

def df_to_array(df):
  df_list = []
  for i in range(len(df[0])):
    df_list.append(df[0][i])
  df_list = np.array(df_list)
  df_list.reshape(len(df[0][0]),-1)
  return df_list

def normalize_data(data):
  eps = 1e-5
  N = data.shape[0]
  mu = 1/N*np.sum(data,axis=0)
  sigma = 1/N*np.sum((data-mu)**2,axis=0)
  normalized_data = (data-mu)/(np.sqrt(sigma+eps))
  return normalized_data

In [ ]:
## DEFINE THE SETS
X_train = df_to_array(train_df)
X_valid = df_to_array(valid_df)
X_test = df_to_array(test_df)

print("Train shape: ", X_train.shape)
print("Valid shape: ", X_valid.shape)
print("Test shape: ", X_test.shape)

Train shape:  (103683, 100)
Valid shape:  (55832, 100)
Test shape:  (63542, 100)


In [ ]:
## DEFINE THE LABELS FOR "TOXIC"
y_train_toxic = train_df[1]
y_valid_toxic = valid_df[1]
y_test_toxic = test_df[1]

In [ ]:
toxic_scores, f1_toxic = basic_classifier_tester(X_train,y_train_toxic,X_valid,y_valid_toxic)

LogisticRegression:
              precision    recall  f1-score   support

           0       0.98      0.42      0.59     50435
           1       0.14      0.92      0.25      5397

    accuracy                           0.47     55832
   macro avg       0.56      0.67      0.42     55832
weighted avg       0.90      0.47      0.56     55832

Confusion Matrix
[[21277 29158]
 [  454  4943]]
------------------------------------------------------
BernoulliNB:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     50435
           1       0.52      0.34      0.41      5397

    accuracy                           0.91     55832
   macro avg       0.73      0.65      0.68     55832
weighted avg       0.89      0.91      0.90     55832

Confusion Matrix
[[48746  1689]
 [ 3556  1841]]
------------------------------------------------------


In [ ]:
## DEFINE THE LABELS FOR "SEVERE TOXIC"
y_train_sevtoxic = train_df[2]
y_valid_sevtoxic = valid_df[2]
y_test_sevtoxic = test_df[2]

In [ ]:
sev_toxic_scores, f1_sevtoxic = basic_classifier_tester(X_train,y_train_sevtoxic,X_valid,y_valid_sevtoxic)

LogisticRegression:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     55275
           1       0.06      0.03      0.04       557

    accuracy                           0.99     55832
   macro avg       0.52      0.51      0.52     55832
weighted avg       0.98      0.99      0.98     55832

Confusion Matrix
[[54985   290]
 [  539    18]]
------------------------------------------------------
BernoulliNB:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     55275
           1       0.36      0.15      0.21       557

    accuracy                           0.99     55832
   macro avg       0.68      0.57      0.60     55832
weighted avg       0.99      0.99      0.99     55832

Confusion Matrix
[[55124   151]
 [  472    85]]
------------------------------------------------------


In [ ]:
## DEFINE THE LABELS FOR "OBSCENE"
y_train_obscene = train_df[3]
y_valid_obscene = valid_df[3]
y_test_obscene = test_df[3]

In [ ]:
obscene_scores, f1_obscene = basic_classifier_tester(X_train,y_train_obscene,X_valid,y_valid_obscene)

LogisticRegression:
              precision    recall  f1-score   support

           0       0.99      0.53      0.69     53021
           1       0.09      0.86      0.16      2811

    accuracy                           0.55     55832
   macro avg       0.54      0.70      0.43     55832
weighted avg       0.94      0.55      0.66     55832

Confusion Matrix
[[28236 24785]
 [  396  2415]]
------------------------------------------------------
BernoulliNB:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     53021
           1       0.53      0.21      0.30      2811

    accuracy                           0.95     55832
   macro avg       0.74      0.60      0.64     55832
weighted avg       0.94      0.95      0.94     55832

Confusion Matrix
[[52494   527]
 [ 2227   584]]
------------------------------------------------------


In [ ]:
## DEFINE THE LABELS FOR "THREAT"
y_train_threat = train_df[4]
y_valid_threat = valid_df[4]
y_test_threat = test_df[4]

In [ ]:
threat_scores, f1_threat = basic_classifier_tester(X_train,y_train_threat,X_valid,y_valid_threat)

LogisticRegression:
              precision    recall  f1-score   support

           0       0.99      0.43      0.59     52825
           1       0.08      0.93      0.15      3007

    accuracy                           0.45     55832
   macro avg       0.54      0.68      0.37     55832
weighted avg       0.94      0.45      0.57     55832

Confusion Matrix
[[22458 30367]
 [  225  2782]]
------------------------------------------------------
BernoulliNB:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     52825
           1       0.55      0.24      0.34      3007

    accuracy                           0.95     55832
   macro avg       0.75      0.62      0.66     55832
weighted avg       0.94      0.95      0.94     55832

Confusion Matrix
[[52226   599]
 [ 2275   732]]
------------------------------------------------------


In [ ]:
## DEFINE THE LABELS FOR "INSULT"
y_train_insult = train_df[5]
y_valid_insult = valid_df[5]
y_test_insult = test_df[5]

In [ ]:
insult_scores, f1_insult = basic_classifier_tester(X_train,y_train_insult,X_valid,y_valid_insult)

LogisticRegression:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     55668
           1       0.02      0.23      0.04       164

    accuracy                           0.97     55832
   macro avg       0.51      0.60      0.51     55832
weighted avg       0.99      0.97      0.98     55832

Confusion Matrix
[[54099  1569]
 [  126    38]]
------------------------------------------------------
BernoulliNB:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     55668
           1       0.06      0.07      0.07       164

    accuracy                           0.99     55832
   macro avg       0.53      0.53      0.53     55832
weighted avg       0.99      0.99      0.99     55832

Confusion Matrix
[[55488   180]
 [  152    12]]
------------------------------------------------------


In [ ]:
## DEFINE THE LABELS FOR "IDENTITY HATE"
y_train_iden_hate = train_df[6]
y_valid_iden_hate = valid_df[6]
y_test_iden_hate = test_df[6]

In [ ]:
identity_hate_scores, f1_iden_hate = basic_classifier_tester(X_train,y_train_iden_hate,X_valid,y_valid_iden_hate)

LogisticRegression:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     55335
           1       0.03      0.18      0.06       497

    accuracy                           0.95     55832
   macro avg       0.51      0.57      0.52     55832
weighted avg       0.98      0.95      0.96     55832

Confusion Matrix
[[52819  2516]
 [  407    90]]
------------------------------------------------------
BernoulliNB:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     55335
           1       0.14      0.15      0.14       497

    accuracy                           0.98     55832
   macro avg       0.56      0.57      0.57     55832
weighted avg       0.98      0.98      0.98     55832

Confusion Matrix
[[54857   478]
 [  421    76]]
------------------------------------------------------


In [ ]:
## AVERAGE F1 SCORES

n_classes = 6
avg_LR = (f1_toxic[0]+f1_sevtoxic[0]+f1_obscene[0]+f1_threat[0]+f1_insult[0]+f1_iden_hate[0])/n_classes
avg_NB = (f1_toxic[1]+f1_sevtoxic[1]+f1_obscene[1]+f1_threat[1]+f1_insult[1]+f1_iden_hate[1])/n_classes

print("F1 score LR: ", avg_LR)
print("F1 score NB: ", avg_NB)

F1 score LR:  0.1179435551697302
F1 score NB:  0.24569370507222302


### Can we do better on class imbalance: LR with weights and GaussianNB with normalized data

In [ ]:
## CHECK CLASS DISTRIBUTION

d = {1:"Toxic",2:"Severe Toxic",3:"Obscene",4:"Threat",5:"Insult",6:"Identity hate"}

for k in d.keys():
  print(d[k] + ": ")
  print(train_df[k].value_counts()/train_df.shape[0])
  print("----------------------")

Toxic: 
0    0.904546
1    0.095454
Name: 1, dtype: float64
----------------------
Severe Toxic: 
0    0.989989
1    0.010011
Name: 2, dtype: float64
----------------------
Obscene: 
0    0.95114
1    0.04886
Name: 3, dtype: float64
----------------------
Threat: 
0    0.947513
1    0.052487
Name: 4, dtype: float64
----------------------
Insult: 
0    0.996972
1    0.003028
Name: 5, dtype: float64
----------------------
Identity hate: 
0    0.991243
1    0.008757
Name: 6, dtype: float64
----------------------


In [ ]:
## DEFINE WEIGHTS TO PASS TO LOGISTIC REGRESSION

w_toxic = {0:10, 1:90}
w_sev_toxic = {0:1, 1:99}
w_obscene = {0:5, 1:95}
w_threat = {0:5, 1:95}
w_insult = {0:0.3, 1:99.7}
w_iden_hate = {0:0.8, 1:99.2}

weights = [w_toxic,w_sev_toxic,w_obscene,w_threat,w_insult,w_iden_hate]

In [ ]:
## NORMALIZE THE DATASETS FOR THE GAUSSIANNB

X_train_norm = normalize_data(X_train)
X_valid_norm = normalize_data(X_valid)
X_test_norm = normalize_data(X_test)

In [ ]:
## USE THE WEIGHTED LR AND GAUSSIANNB

y_train = [y_train_toxic,y_train_sevtoxic,y_train_obscene,y_train_threat,y_train_insult,y_train_iden_hate]
y_valid = [y_valid_toxic,y_valid_sevtoxic,y_valid_obscene,y_valid_threat,y_valid_insult,y_valid_iden_hate]
y_test = [y_test_toxic,y_test_sevtoxic,y_test_obscene,y_test_threat,y_test_insult,y_test_iden_hate]

use_gaussian = True
final_scores = []
final_f1 = []

for i in range(len(y_train)):
  print(">>>>>>>>>>>>>> ", d[i+1])
  w = weights[i]
  scores, f1 = basic_classifier_tester(X_train_norm,y_train[i],X_valid_norm,y_valid[i],w,use_gaussian)
  final_scores.append(scores)
  final_f1.append(f1)

>>>>>>>>>>>>>>  Toxic
LogisticRegression:
              precision    recall  f1-score   support

           0       0.95      0.81      0.87     50435
           1       0.24      0.56      0.34      5397

    accuracy                           0.79     55832
   macro avg       0.59      0.69      0.61     55832
weighted avg       0.88      0.79      0.82     55832

Confusion Matrix
[[40966  9469]
 [ 2354  3043]]
------------------------------------------------------
GaussianNB:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93     50435
           1       0.37      0.37      0.37      5397

    accuracy                           0.88     55832
   macro avg       0.65      0.65      0.65     55832
weighted avg       0.88      0.88      0.88     55832

Confusion Matrix
[[47055  3380]
 [ 3374  2023]]
------------------------------------------------------
>>>>>>>>>>>>>>  Severe Toxic
LogisticRegression:
              precision    recall  f1

In [ ]:
## AVERAGE FINAL F1 SCORES

f1_LR_final = 0
f1_NB_final = 0

for i in range(len(final_f1)):
  f1_LR_final += final_f1[i][0]
  f1_NB_final += final_f1[i][1]

print("Final F1 score LR: ", f1_LR_final/n_classes)
print("Final F1 score NB: ", f1_NB_final/n_classes)

Final F1 score LR:  0.16571803971384222
Final F1 score NB:  0.23760113307383054


We will now try to use the best performing model (BernoulliNB) to implement a naive ranking model.

## Simplified scoring model

In [ ]:
## FUNCTIONS

def bernoulliClassifier(X_train,Y_train,X_test):
  bn = BernoulliNB()
  bn.fit(X_train, Y_train)
  predictions = bn.predict(X_test)
  prob = bn.predict_proba(X_test)
  return list(predictions), list(prob)

def remove_empty_tokenized_values(data):
  return data.loc[data["comment_tokenize"].str.len() != 0]

In [ ]:
## GET THE TRAINING SET PREDICTIONS

predictions = []
proba = []

for i in range(len(y_train)):
  print(">>>>>>>>>>>>>> ", d[i+1])
  pred, prob = bernoulliClassifier(X_train,y_train[i],X_test)
  predictions.append(pred)
  proba.append(prob)

>>>>>>>>>>>>>>  Toxic
>>>>>>>>>>>>>>  Severe Toxic
>>>>>>>>>>>>>>  Obscene
>>>>>>>>>>>>>>  Threat
>>>>>>>>>>>>>>  Insult
>>>>>>>>>>>>>>  Identity hate


In [ ]:
## CLEAN THE COMMENT SET (remove the empty tokenizations so that the sizes match)

comment_test_data = remove_empty_tokenized_values(comment_test_data) 

In [ ]:
## GET THE WEIGHTS FOR EVERY CLASS

for i in range(len(y_train)):
  print(">>>>>>>>>>>>>> ", d[i+1])
  w = weights[i]
  print(w[0])

>>>>>>>>>>>>>>  Toxic
10
>>>>>>>>>>>>>>  Severe Toxic
1
>>>>>>>>>>>>>>  Obscene
5
>>>>>>>>>>>>>>  Threat
5
>>>>>>>>>>>>>>  Insult
0.3
>>>>>>>>>>>>>>  Identity hate
0.8


In [ ]:
## COMPUTE SCORES BASED ON WEIGHTS

def get_simplified_score(label_tag, probabilities):

  if label_tag[4] == 1: #insult
    score = 9 + probabilities[4][1]
  elif label_tag[5] == 1: #identity hate
    score = 8 + probabilities[5][1]
  elif label_tag[1] == 1: #severe toxic
    score = 7 + probabilities[1][1]
  elif label_tag[2] == 1 or label_tag[3] == 1: #threat or obscene
    score1 = 5 + 2*probabilities[2][1]
    score2 = 5 + 2*probabilities[3][1]
    score = max(score1,score2)
  elif label_tag[0] == 1: # toxic
    score = 5*probabilities[0][1]
  else: #NON toxic
    score = 0

  return score

def get_column_tag(data_result, ex_idx):
  result = []
  for i in range(6):
    result.append(data_result[i][ex_idx])
  return result


In [ ]:
## GET THE SCORES

N = len(comment_test_data["toxic"])
scores = []

for i in range(N):
  s = get_column_tag(predictions, i)
  p = get_column_tag(proba, i)
  scores.append(get_simplified_score(s,p))


In [ ]:
## EXAMPLES

score_list = [195,216,433,28,274]

for i in score_list: 
  print('Scores: {}'.format(round(scores[i],4)).ljust(20) + 'Comment: ' + comment_test_data["comment"].iloc[i])

Scores: 0           Comment: i think the motd is a waste of time
Scores: 0           Comment: thank god i am an atheist
Scores: 3.5643      Comment: kevin bravo is a faggot
Scores: 4.0478      Comment: fuck off you anti semitic cunt
Scores: 6.9839      Comment: karl tearle is a mop haired twat


In [ ]:
for i in range(500,700): 
  comment = comment_test_data["comment"].iloc[i]
  truncated_comment = (comment[:60] + ' ...') if len(comment) > 60 else comment
  print(str(i) + " " + 'Scores: {}'.format(round(scores[i],4)).ljust(20) + 'Comment: ' + truncated_comment)

500 Scores: 0           Comment: designation ship types function the introductory table of va ...
501 Scores: 0           Comment: interwiki could an admin add de la studio an new interwiki l ...
502 Scores: 0           Comment: of course it is more complex than that i would suggest every ...
503 Scores: 0           Comment: fjtfbytrhdfshf nbaahjddnnxcxdhdhv h fthcvyd
504 Scores: 0           Comment: music is there a name for a fear of musical instruments im s ...
505 Scores: 0           Comment: also there is a separate article terrorism in kashmir for th ...
506 Scores: 0           Comment: proposed georgia move as a past participant in the discussio ...
507 Scores: 0           Comment: wikipedia is for dickshits like you retarded wikpedia is gay
508 Scores: 0           Comment: aww it must have been out on its own for a while and caught  ...
509 Scores: 0           Comment: i just left some further comments at the ga review it is the ...
510 Scores: 0           Comment: frankly i wo